## Assignment 1

### Step 1: Import Gurobi

In [1]:
import gurobipy as gp
from gurobipy import GRB
import random
import csv
import pprint

In [2]:
warehouses = {
    0:  "thunder bay",
    1:  "sudbury",
    2:  "ottawa",
    3:  "toronto",
    4:  "kingston",
    5:  "windsor",
    6:  "niagara falls",
    7:  "london"
}

retailers = {
    0:  "vaughan",
    1:  "brampton",
    2:  "toronto",
    3:  "mississauga",
    4:  "kitchener",
    5:  "hamilton",
    6:  "leamington",
    7:  "chatham kent",
    8:  "sarnia",
    9:  "stratford",
    10: "bellville",
    11: "peterborough",
    12: "orillia",
    13: "collingwood",
    14: "oshawa",
    15: "milton",
    16: "brantford",
    17: "st catherines",
    18: "welland",
    19: "woodstock"
}

In [3]:
# # Debug Cell
# print(distances)
# for m in range(len(distances)):
#     for n in range(len(distances[0])):
#         print(f"{warehouses[m]} -> {retailers[n]}: {distances[m][n]} km" )

---

## a) Base Mathematical Model

### Define Model

In [4]:
m = gp.Model()

### Sets

In [5]:
P = ['Barrie', 'Cambridge']

W = [w for w in warehouses.values()]

R = [r for r in retailers.values()]

### Parameters

In [6]:
# Plant capacity
Q_p = [1500, 1500]

# Warehouse Capacity
Q_w = [200, 400, 200, 800, 800, 400, 300, 600]

# Warehouse Cost
C_w = [90, 110, 150, 150, 90, 110, 110, 150]

# Transportation Cost
t = 0.187

# Retailer Demand
random.seed(20726975) # Matt's student ID
d_r = [random.randint(10000,75000) for r in range(len(R))]

# Distances
distances = []

with open ('distances.csv', newline='') as f:
    reader = csv.reader(f, delimiter=' ')
    for row in reader:
        row = [float(distance) for distance in ', '.join(row).split(',')]
        distances.append(row)

D = [
        [1441, 458, 483, 99.6, 343, 278, 129, 102],
        [1290, 308, 460, 103, 332, 425, 197, 250]
]

# Carbon cap in tonnes
carbon_cap = 805

# Per km emissions in tonnes
transportation_emissions_rate = 0.00167 / 1000

# Emissions per plant p
plant_emissions= [449, 268]

# Emissions per warehouse w
warehouse_emissions = [65, 80, 95, 95, 65, 80, 80, 95]

# Big M
big_m = 1.37 * 10 ** 8

### Decision Variables

In [7]:
# Number of products transported from plant p to warehouse w
x1 = {}
for p in range(len(P)):
    for w in range(len(W)):
        x1[p,w] = m.addVar(lb=0.0, obj=D[p][w]*t, vtype=GRB.INTEGER, name="x1_pw_"+str(p)+str(w))

# Number of products transported from warehouse w to retailer r
x2 = {}
for w in range(len(W)):
    for r in range(len(R)):
        x2[w,r] = m.addVar(lb=0.0, obj=distances[w][r]*d_r[r]*t, vtype=GRB.BINARY, name="x2_wr_"+str(w)+str(r))

# Binary variable for whether or not warehouse w is open
y = {}
for w in range(len(W)):
    y[w] = m.addVar(obj=C_w[w]*1000, vtype=GRB.BINARY, name="y_"+str(w))

# Binary variable for whether or not plant p was used
plant_used = {}
for p in range(len(P)):
    plant_used[p] = m.addVar(lb=0.0, vtype=GRB.BINARY, name="plant_used_"+str(p))
    
    

### Objective Function

In [8]:
m.modelSense = GRB.MINIMIZE

### Constraints

In [9]:
# Single Allocation Constraint
for r in range(len(R)):
    single_alloc = 0
    for w in range(len(W)):
        single_alloc += x2[w,r]
    m.addConstr(single_alloc, GRB.EQUAL, 1)

# Plant Production Capacity Constraint
for p in range(len(P)):
    cap_prod = 0
    for w in range(len(W)):
        cap_prod += x1[p,w]
    m.addConstr(cap_prod, GRB.LESS_EQUAL, Q_p[p]*1000)
    
# Warehouse Incoming Capacity Constraint
for w in range(len(W)):
    cap_in = 0
    for p in range(len(P)):
        cap_in += x1[p,w]
    m.addConstr(cap_in, GRB.LESS_EQUAL, Q_w[w]*1000*y[w])

# Warehouse Flow Constraint
for w in range(len(W)):
    flow_in = 0
    flow_out = 0
    for p in range(len(P)):
        flow_in += x1[p,w]
    for r in range(len(R)):
        # binary * demand b.c. single alloc
        flow_out += x2[w,r]*d_r[r]
    m.addConstr(flow_in, GRB.EQUAL, flow_out)
    
# Carbon Cap Constraint - warehouse + plant + total distance travelled
total_carbon = 0
# # Emissions for all the warehouses that are open
for w in range (len(W)):
    total_carbon += y[w] * warehouse_emissions[w]   
# Emissions for transportation between plant and warehouses
for p in range(len(P)):
    for w in range(len(W)):
        total_carbon += x1[p,w] * transportation_emissions_rate * D[p][w]
# Emissions for transportation between warehouses and retailers
for w in range(len(W)):
    for r in range(len(R)):
        total_carbon += x2[w,r] * transportation_emissions_rate * d_r[r] * distances[w][r]
## Emissions for plants, we're using big M to set a binary var to see whether or not things ar efine
for p in range(len(P)):
    shipped = 0
    for w in range(len(W)):
        shipped += x1[p,w]
    m.addConstr(shipped >= plant_used[p])
    m.addConstr(shipped <= big_m * plant_used[p])
    total_carbon += plant_used[p] * plant_emissions[p]
    
m.addConstr(total_carbon, GRB.LESS_EQUAL, carbon_cap)

m.update()

### Solve the Model

In [10]:
m.optimize()

Gurobi Optimizer version 9.0.1 build v9.0.1rc0 (mac64)
Optimize a model with 43 rows, 186 columns and 598 nonzeros
Model fingerprint: 0x6e7de5bd
Variable types: 0 continuous, 186 integer (170 binary)
Coefficient statistics:
  Matrix range     [2e-04, 1e+08]
  Objective range  [2e+01, 2e+07]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+06]
         Consider reformulating model or setting NumericFocus parameter
         to avoid numerical issues.
Presolve removed 2 rows and 0 columns
Presolve time: 0.00s
Presolved: 41 rows, 186 columns, 561 nonzeros
Variable types: 0 continuous, 186 integer (170 binary)

Root relaxation: objective 3.244311e+07, 35 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 3.2443e+07    0    4          - 3.2443e+07      -     -    0s
     0     0 3.3326e+07    0    9          - 3.3326e+07      -     -    0s


In [11]:
# print(y)
# print(x1)
# print(x2)
print(m.getVars())

[<gurobi.Var x1_pw_00 (value -0.0)>, <gurobi.Var x1_pw_01 (value -0.0)>, <gurobi.Var x1_pw_02 (value -0.0)>, <gurobi.Var x1_pw_03 (value -0.0)>, <gurobi.Var x1_pw_04 (value -0.0)>, <gurobi.Var x1_pw_05 (value -0.0)>, <gurobi.Var x1_pw_06 (value -0.0)>, <gurobi.Var x1_pw_07 (value -0.0)>, <gurobi.Var x1_pw_10 (value -0.0)>, <gurobi.Var x1_pw_11 (value 0.0)>, <gurobi.Var x1_pw_12 (value 0.0)>, <gurobi.Var x1_pw_13 (value 768614.0)>, <gurobi.Var x1_pw_14 (value 0.0)>, <gurobi.Var x1_pw_15 (value 0.0)>, <gurobi.Var x1_pw_16 (value 0.0)>, <gurobi.Var x1_pw_17 (value 161588.0)>, <gurobi.Var x2_wr_00 (value -0.0)>, <gurobi.Var x2_wr_01 (value -0.0)>, <gurobi.Var x2_wr_02 (value -0.0)>, <gurobi.Var x2_wr_03 (value -0.0)>, <gurobi.Var x2_wr_04 (value -0.0)>, <gurobi.Var x2_wr_05 (value -0.0)>, <gurobi.Var x2_wr_06 (value -0.0)>, <gurobi.Var x2_wr_07 (value -0.0)>, <gurobi.Var x2_wr_08 (value -0.0)>, <gurobi.Var x2_wr_09 (value -0.0)>, <gurobi.Var x2_wr_010 (value -0.0)>, <gurobi.Var x2_wr_011 (

In [12]:
for myVars in m.getVars():
#     print(myVars)
    print('%s %g' % (myVars.varName, myVars.x))

x1_pw_00 -0
x1_pw_01 -0
x1_pw_02 -0
x1_pw_03 -0
x1_pw_04 -0
x1_pw_05 -0
x1_pw_06 -0
x1_pw_07 -0
x1_pw_10 -0
x1_pw_11 0
x1_pw_12 0
x1_pw_13 768614
x1_pw_14 0
x1_pw_15 0
x1_pw_16 0
x1_pw_17 161588
x2_wr_00 -0
x2_wr_01 -0
x2_wr_02 -0
x2_wr_03 -0
x2_wr_04 -0
x2_wr_05 -0
x2_wr_06 -0
x2_wr_07 -0
x2_wr_08 -0
x2_wr_09 -0
x2_wr_010 -0
x2_wr_011 -0
x2_wr_012 -0
x2_wr_013 -0
x2_wr_014 -0
x2_wr_015 -0
x2_wr_016 -0
x2_wr_017 -0
x2_wr_018 -0
x2_wr_019 -0
x2_wr_10 -0
x2_wr_11 -0
x2_wr_12 -0
x2_wr_13 -0
x2_wr_14 -0
x2_wr_15 -0
x2_wr_16 -0
x2_wr_17 -0
x2_wr_18 -0
x2_wr_19 -0
x2_wr_110 -0
x2_wr_111 -0
x2_wr_112 0
x2_wr_113 0
x2_wr_114 -0
x2_wr_115 -0
x2_wr_116 -0
x2_wr_117 -0
x2_wr_118 -0
x2_wr_119 -0
x2_wr_20 -0
x2_wr_21 -0
x2_wr_22 -0
x2_wr_23 -0
x2_wr_24 -0
x2_wr_25 -0
x2_wr_26 -0
x2_wr_27 -0
x2_wr_28 -0
x2_wr_29 -0
x2_wr_210 0
x2_wr_211 0
x2_wr_212 -0
x2_wr_213 -0
x2_wr_214 -0
x2_wr_215 -0
x2_wr_216 -0
x2_wr_217 -0
x2_wr_218 -0
x2_wr_219 -0
x2_wr_30 1
x2_wr_31 1
x2_wr_32 1
x2_wr_33 1
x2_wr_34 1
x2_w

In [13]:
print(total_carbon)

<gurobi.LinExpr: 65.0 y_0 + 80.0 y_1 + 95.0 y_2 + 95.0 y_3 + 65.0 y_4 + 80.0 y_5 + 80.0 y_6 + 95.0 y_7 + 0.00240647 x1_pw_00 + 0.00076486 x1_pw_01 + 0.00080661 x1_pw_02 + 0.000166332 x1_pw_03 + 0.00057281 x1_pw_04 + 0.00046426000000000003 x1_pw_05 + 0.00021543000000000002 x1_pw_06 + 0.00017034 x1_pw_07 + 0.0021543 x1_pw_10 + 0.0005143600000000001 x1_pw_11 + 0.0007682 x1_pw_12 + 0.00017201 x1_pw_13 + 0.00055444 x1_pw_14 + 0.0007097500000000001 x1_pw_15 + 0.00032899000000000003 x1_pw_16 + 0.0004175 x1_pw_17 + 145.252926 x2_wr_00 + 141.54409815000002 x2_wr_01 + 63.17754956000001 x2_wr_02 + 165.91314396 x2_wr_03 + 180.24964640000002 x2_wr_04 + 123.20220592000001 x2_wr_05 + 57.85926422 x2_wr_06 + 67.9888396 x2_wr_07 + 120.64833504 x2_wr_08 + 93.74623824000001 x2_wr_09 + 133.97451253 x2_wr_010 + 100.46781456 x2_wr_011 + 120.46396537000001 x2_wr_012 + 48.185351680000004 x2_wr_013 + 97.89755430000001 x2_wr_014 + 167.17039845000002 x2_wr_015 + 117.3169656 x2_wr_016 + 32.1276938 x2_wr_017 + 73.2